In [5]:
import numpy as np
import pandas as pd
import os
import shutil
import sklearn.decomposition
# import qtl_config_utils
import sys

In [6]:
outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/timepoint_D52/input_files_untreated/celltype_DA/extra_runs_addressing_reviews/"

In [7]:
# general set up
genotypes_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered'
kinship_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered.rel'
annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/ensembl_gene_id_annos.tsv'
#annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/hgnc_symbol_annos.tsv'
chunk_file = '/nfs/leia/research/stegle/mjbonder/ChunkFiles/Ensembl_75_Limix_Annotation_FC_Gene_step100.txt'
n_pcs = 15
base_dir = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/timepoint_D52/'

In [8]:
sample_mapping_file = os.path.join(outdir, 'samplemapping.tsv')
phenotype_file = os.path.join(outdir, 'phenotypes.tsv')
covariates_file = os.path.join(outdir, 'covariates.tsv')
# noise_matrix_file = os.path.join(outdir, 'noise_matrix.tsv')
# config_file = os.path.join(outdir, 'qtl_config.yaml')

In [9]:
# eQTL discovery parameters
#number of top expressed genes to test
n_genes = 50000

In [10]:
if not os.path.exists(outdir):
    os.makedirs(outdir, exist_ok=True)

In [11]:
mydir = '/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/'
## DA
D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-pool_id-celltype-treatment.mean.tsv'

In [12]:
phenotype_input_file = mydir + D52

In [13]:
# phenotype_input_file = sys.argv[1]
# outdir = sys.argv[2]
# if len(sys.argv)>=4:
#     subset_field = sys.argv[3]
#     selected_subset = sys.argv[4]
# else:
#     subset_field = None
#     selected_subset = None

In [14]:
#load map HGNC to ensembl
mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')

In [15]:
# process phenotype dataframe (untreated)
groupedby_df = pd.read_csv(phenotype_input_file, sep='\t')
groupedby_df = groupedby_df.query('treatment=="NONE"')

In [16]:
phenotype_df = groupedby_df.copy()

In [17]:
phenotype_df.head()

,donor_id,pool_id,celltype,treatment,n_cells,AP006222.2,RP11-206L10.3,RP11-206L10.9,LINC00115,FAM41C,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,pool10,DA,NONE,66,0.373148,0.010521,0.034687,0.012997,0.014373,...,3.452707,1.843407,0.599258,3.306375,2.241297,0.292286,3.334886,0.049472,0.382724,0.076166
2,HPSI0114i-eipl_1,pool1,DA,NONE,189,0.362936,0.006138,0.015826,0.026636,0.023932,...,3.618761,2.037559,0.564256,3.442609,2.081173,0.198851,3.564958,0.000000,0.000000,0.093761
4,HPSI0114i-fikt_3,pool14,DA,NONE,26,0.269007,0.000000,0.062557,0.000000,0.000000,...,3.209428,2.040483,0.669133,3.445064,2.108070,0.153443,3.353447,0.032703,0.161917,0.102277
6,HPSI0114i-kolf_2,pool10,DA,NONE,40,0.391692,0.000000,0.018476,0.065576,0.067277,...,3.440805,1.465916,0.297830,3.218524,2.225487,0.130202,2.929145,0.081922,0.045592,0.037953
8,HPSI0114i-kolf_2,pool15,DA,NONE,507,0.330450,0.008953,0.019507,0.014054,0.018184,...,3.714630,1.703519,0.547933,3.542038,2.397292,0.151074,3.211312,0.053037,0.033045,0.002556


In [18]:
# should only be cells from one celltype
assert(len(phenotype_df['celltype'].drop_duplicates())==1)

In [19]:
#create merged index
phenotype_df['index'] = phenotype_df['donor_id']+"."+phenotype_df['pool_id']
cell_count_ds = phenotype_df['n_cells']
cell_count_ds.index = phenotype_df['index'].tolist()

In [20]:
#take donor and pool cols for samplemapping df
samplemapping_df = phenotype_df[['donor_id','index']]
samplemapping_df.head()
# samplemapping_df.to_csv(sample_mapping_file, sep='\t', index=False, header=False)

,donor_id,index
0,HPSI0114i-bezi_3,HPSI0114i-bezi_3.pool10
2,HPSI0114i-eipl_1,HPSI0114i-eipl_1.pool1
4,HPSI0114i-fikt_3,HPSI0114i-fikt_3.pool14
6,HPSI0114i-kolf_2,HPSI0114i-kolf_2.pool10
8,HPSI0114i-kolf_2,HPSI0114i-kolf_2.pool15


In [21]:
#reorganise to just be expression data indexed by the merged index
cols_to_drop = list(set(phenotype_df.columns) & {'donor_id','celltype','n_cells','treatment','pool_id'})
phenotype_df = phenotype_df.drop(cols_to_drop, axis=1).set_index('index')
phenotype_df = phenotype_df.transpose()
phenotype_df.head()

index,HPSI0114i-bezi_3.pool10,HPSI0114i-eipl_1.pool1,HPSI0114i-fikt_3.pool14,HPSI0114i-kolf_2.pool10,HPSI0114i-kolf_2.pool15,HPSI0114i-lexy_2.pool10,HPSI0114i-lexy_2.pool17,HPSI0114i-posc_1.pool10,HPSI0114i-rozh_4.pool17,HPSI0114i-vass_1.pool17,...,HPSI1113i-qolg_1.pool9,HPSI1113i-qolg_1.pool16,HPSI1113i-qorq_1.pool17,HPSI1113i-vuud_2.pool5,HPSI1114i-kuul_1.pool6,HPSI1114i-kuul_1.pool7,HPSI1114i-ualf_6.pool6,HPSI1114i-zuuy_4.pool13,HPSI1213i-hehd_1.pool4,HPSI1213i-hehd_1.pool5
AP006222.2,0.373148,0.362936,0.269007,0.391692,0.330450,0.292542,0.202281,0.376424,0.343145,0.324519,...,0.183824,0.464616,0.276859,0.375452,0.317608,0.358510,0.364334,0.605280,0.306149,0.370794
RP11-206L10.3,0.010521,0.006138,0.000000,0.000000,0.008953,0.018228,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.016256,0.008668,0.013400,0.000000,0.010513,0.000000,0.000000,0.007563,0.003472
RP11-206L10.9,0.034687,0.015826,0.062557,0.018476,0.019507,0.014019,0.021834,0.014933,0.015792,0.000000,...,0.000000,0.004781,0.026287,0.013560,0.000000,0.011588,0.000000,0.025472,0.004336,0.031014
LINC00115,0.012997,0.026636,0.000000,0.065576,0.014054,0.015915,0.018512,0.055751,0.037459,0.022826,...,0.000000,0.006654,0.013481,0.011352,0.022266,0.006758,0.000000,0.122520,0.007679,0.008590
FAM41C,0.014373,0.023932,0.000000,0.067277,0.018184,0.000000,0.000000,0.017594,0.000000,0.029933,...,0.000000,0.023529,0.008824,0.012704,0.009125,0.006559,0.056234,0.063463,0.005390,0.004907


In [22]:
gene_names = phenotype_df.index

In [23]:
#number of top expressed genes
selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
phenotype_df = phenotype_df.loc[selected_genes, :]

In [24]:
phenotype_list = phenotype_df.index
mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
mapping_df = mapping_df.set_index('hgnc_symbol')

In [25]:
# limit only to hgnc symbols that map to ensembl gene IDs
phenotype_df = phenotype_df.loc[mapping_df.index,:]
phenotype_df.index = mapping_df['ensembl_gene_id']
phenotype_df.head()

index,HPSI0114i-bezi_3.pool10,HPSI0114i-eipl_1.pool1,HPSI0114i-fikt_3.pool14,HPSI0114i-kolf_2.pool10,HPSI0114i-kolf_2.pool15,HPSI0114i-lexy_2.pool10,HPSI0114i-lexy_2.pool17,HPSI0114i-posc_1.pool10,HPSI0114i-rozh_4.pool17,HPSI0114i-vass_1.pool17,...,HPSI1113i-qolg_1.pool9,HPSI1113i-qolg_1.pool16,HPSI1113i-qorq_1.pool17,HPSI1113i-vuud_2.pool5,HPSI1114i-kuul_1.pool6,HPSI1114i-kuul_1.pool7,HPSI1114i-ualf_6.pool6,HPSI1114i-zuuy_4.pool13,HPSI1213i-hehd_1.pool4,HPSI1213i-hehd_1.pool5
ensembl_gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000261657,0.199801,0.170441,0.135957,0.119144,0.218072,0.189537,0.106559,0.074559,0.247768,0.178011,...,0.229570,0.157384,0.180572,0.158088,0.149935,0.170819,0.245467,0.090187,0.205873,0.259147
ENSG00000227640,0.000000,0.020137,0.000000,0.017075,0.009690,0.013281,0.110367,0.000000,0.053130,0.050767,...,0.061906,0.024022,0.096830,0.018867,0.000000,0.007348,0.000000,0.000000,0.041669,0.000000
ENSG00000247400,0.131556,0.121530,0.340275,0.134915,0.121546,0.119536,0.100597,0.062595,0.091913,0.117875,...,0.089358,0.095934,0.074252,0.166735,0.089448,0.126966,0.245197,0.210142,0.166437,0.147348
ENSG00000228889,0.019465,0.047815,0.068368,0.016302,0.026738,0.031981,0.000000,0.026038,0.011520,0.035781,...,0.176883,0.015375,0.023026,0.023460,0.035566,0.041129,0.041154,0.034466,0.045126,0.040083
ENSG00000261125,0.069042,0.052439,0.033319,0.030723,0.047611,0.000000,0.016495,0.083186,0.066110,0.070324,...,0.000000,0.037210,0.028379,0.059444,0.047389,0.039217,0.106279,0.025472,0.049448,0.087586


In [26]:
phenotype_df.to_csv(phenotype_file, sep='\t')

In [27]:
pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])
pc_df.head(2)
# pc_df.to_csv(covariates_file, sep='\t')

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15
index,,,,,,,,,,,,,,,
HPSI0114i-bezi_3.pool10,0.947558,-4.288377,-1.133996,-0.015529,-3.081061,2.084639,-2.724498,1.606894,1.072142,1.025266,-2.400403,0.763368,0.357169,0.396405,-0.055089
HPSI0114i-eipl_1.pool1,-1.511635,1.193321,0.319396,3.146543,-0.294582,-2.233682,0.244273,1.788409,-0.085921,0.481978,-1.455914,-0.580990,-0.495282,0.432955,0.402872


In [28]:
cov_df = groupedby_df.copy()
cov_df = cov_df.drop(gene_names, axis=1)
cov_df['index'] = cov_df['donor_id']+"."+cov_df['pool_id']
# cols_to_drop = list({'donor_id','celltype','treatment'})
cols_to_drop = list({'donor_id','celltype','treatment','n_cells'})
cov_df = cov_df.drop(cols_to_drop, axis=1).set_index('index')
cov_df.head()

,pool_id
index,
HPSI0114i-bezi_3.pool10,pool10
HPSI0114i-eipl_1.pool1,pool1
HPSI0114i-fikt_3.pool14,pool14
HPSI0114i-kolf_2.pool10,pool10
HPSI0114i-kolf_2.pool15,pool15


In [30]:
num_rows = cov_df.shape[0]
pool_list = cov_df['pool_id'].unique()
pool_membership = [ np.reshape((cov_df['pool_id'].values == elem).astype(np.float64), (num_rows,1)) for elem in pool_list ]
design_matrix_pool = np.hstack(tuple(pool_membership))
design_df = pd.DataFrame(design_matrix_pool, index=cov_df.index, columns=pool_list)
# design_df['n_cells'] = cov_df['n_cells']
design_df.head()

,pool10,pool1,pool14,pool15,pool17,pool6,pool7,pool4,pool8,pool2,pool3,pool5,pool9,pool16,pool11,pool13
index,,,,,,,,,,,,,,,,
HPSI0114i-bezi_3.pool10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-eipl_1.pool1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-fikt_3.pool14,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2.pool10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HPSI0114i-kolf_2.pool15,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
design_df.to_csv(covariates_file, sep='\t')

In [27]:
# noise_scaling_vector = [1/float(x) for x in cell_count_ds.tolist()]
# noise_matrix = np.diag(noise_scaling_vector)
# noise_matrix_df = pd.DataFrame(data=noise_matrix, index=cell_count_ds.index, columns=cell_count_ds.index)

In [28]:
# noise_matrix_df.to_csv(noise_matrix_file, sep='\t')
# kinship_file = noise_matrix_file

In [29]:
number_of_permutations = '1000'
minor_allele_frequency = '0.05'
hwe = '0.000001'
call_rate = '1'
window_size = '250000'
block_size = '15000'


config_dict = {'af': annotation_file,
               'pf': phenotype_file,
               'cf': covariates_file,
               'kf': kinship_file,
               'smf': sample_mapping_file,
               'plink': genotypes_file,
               'maf': minor_allele_frequency,
               'hwe': hwe,
               'np': number_of_permutations,
               'cr': call_rate,
               'w': window_size,
               'bs': block_size,
               'chunk_file': chunk_file
}

# write config to a file
qtl_config_utils.write_config(config_dict, config_file)

NameError: name 'qtl_config_utils' is not defined